In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [3]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [4]:
def run_query(conn, table, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    # Create a cursor
    cur = conn.cursor()    
    # Send the query to the database
    cur.execute(query)
    # Extract the results of the query
    results = cur.fetchall()
    # Return the results
    return results

In [5]:
def convert_db_table_to_DF(conn, table):
    # get the names of the attributes in the database table
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
#     print (header_query)
    cols_init = run_query(conn, table, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
#     print(cols)
    # get the records of the table
    content_query = "Select * from " + table
    data = run_query(conn, table, content_query)
    df = pd.DataFrame(data, columns = cols)
    return df

In [6]:
# def main():
database = "chinook.db"

# create a database connection
conn = create_connection(database)    
with conn:
    table = "employees"
    query = "Select * from " + table
    data = run_query(conn, table, query)
    
df = convert_db_table_to_DF(conn, "employees")
df.columns
# if __name__ == '__main__':
#     main()

Index(['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo',
       'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country',
       'PostalCode', 'Phone', 'Fax', 'Email'],
      dtype='object')

# Task 3.1
Question 1:Write python code that uses python connectors to extract all the tables from the database and save them to .csv files

In [7]:
#List with all table names
tableNames = ["media_types", "genres", "playlists", "playlist_track", "tracks", "artists", "invoices", "invoice_items", "albums", "customers", "employees"]
#Empty dictionary to store tables
tableDict = {}

#Loop to store tables in a dictionary and to save each one into a csv
for i in tableNames:
    table = convert_db_table_to_DF(conn, i)
    tableDict[i] = table
    table.to_csv(i + ".csv", index=False)
    
    

In [8]:
#Call a table in tables dictionary
tableDict["playlists"]

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music
5,6,Audiobooks
6,7,Movies
7,8,Music
8,9,Music Videos
9,10,TV Shows


# TASK 3.2
Write the queries that you proposed in Task2 using python. You will need to read the tables that will be used in the queriesinto Pandas Dataframes and write the queries to extract the data from the Dataframes directly (do not use SQL on the original database).

In [9]:
#Defining get tables as function
def get_tables (req):
       #Database Storage Dictionary
    dbDict={}
    #Store each table indicated
    for table in req:
        conn = create_connection(database)    
        with conn:
            dbDict[table] = convert_db_table_to_DF(conn, table)

    return dbDict



# QUERY 1)
QUERY:
"SELECT Title,Name FROM albums LEFT JOIN artists on (artists.ArtistID = albums.ArtistID ) "

In [27]:
database = "chinook.db"

# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["albums","artists"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------


rel = dbDict["albums"].merge(dbDict["artists"], on= "ArtistId", how='left')[["Title","Name"]]
rel


,Title,Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith
...,...,...
342,Respighi:Pines of Rome,Eugene Ormandy
343,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,Mozart: Chamber Music,Nash Ensemble


# QUERY 2)
QUERY:
SELECT COUNT(*) FROM (SELECT composer FROM tracks Where composer ISNULL)

In [11]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["tracks"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------


rel = dbDict["tracks"].isnull().groupby("Composer").count().iloc[1,1]
rel


978

# QUERY 3)
QUERY:
Select distinct tracks.Name FROM tracks EXCEPT Select all tracks.Name FROM tracks join invoice_items on (tracks.trackID  = invoice_items.trackID)

In [28]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["tracks", "invoice_items"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------


part1= dbDict["tracks"]["Name"] 
part1= pd.DataFrame(part1)
part2 = dbDict["tracks"].merge(dbDict["invoice_items"], on= "TrackId", how='inner').groupby("Name").count().index
part2 = pd.DataFrame(part2)
#This merge is equivalent to a EXCEPT in SQL , (using group by to avoid duplicates)
part3 = part1.merge(part2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].groupby("Name").count().index
pd.DataFrame(part3)


,Name
0,"""40"""
1,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525..."
2,#1 Zero
3,(Da Le) Yaleo
4,(I Can't Help) Falling In Love With You
...,...
1364,É Preciso Saber Viver
1365,É Uma Partida De Futebol
1366,É que Nessa Encarnação Eu Nasci Manga
1367,Óia Eu Aqui De Novo


# QUERY 4
QUERY:
Select Name From (SELECT Name,Count(*) as conta FROM artists  join albums on (artists.ArtistID = albums.ArtistID ) Group by (artists.ArtistID) Having (conta =1 ))

In [13]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["artists", "albums"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------


part1=  dbDict["artists"].merge(dbDict["albums"], on= "ArtistId", how='inner').groupby(["Name"]).count()
rel = part1.loc[part1["ArtistId"] == 1].index
rel = pd.DataFrame(rel)
rel

,Name
0,Aaron Copland & London Symphony Orchestra
1,Aaron Goldberg
2,Academy of St. Martin in the Fields & Sir Nevi...
3,Academy of St. Martin in the Fields Chamber En...
4,"Academy of St. Martin in the Fields, John Birc..."
...,...
143,Vinícius De Moraes
144,Wilhelm Kempff
145,Yehudi Menuhin
146,Yo-Yo Ma


# QUERY 5
QUERY:
SELECT SupportRepId, COUNT(*) FROM customers GROUP BY SupportRepId



In [29]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["customers"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------


rel=  dbDict["customers"].groupby("SupportRepId").count()["CustomerId"]
rel


SupportRepId
3    21
4    20
5    18
Name: CustomerId, dtype: int64

# QUERY 6
SELECT min(tracks.Milliseconds), max(tracks.Milliseconds) FROM tracks 



In [42]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["tracks"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------

rel = pd.DataFrame(columns=[["min","max"]])
rel.loc[0,"min"] = dbDict["tracks"]["Milliseconds"].min()
rel.loc[0,"max"] = dbDict["tracks"]["Milliseconds"].max()
rel

,min,max
0,1071,5286953


# QUERY 7
SELECT T.Name ,S.Name, T.Milliseconds FROM tracks T, tracks S WHERE T.Milliseconds = S.Milliseconds  AND T.trackID< S.trackID

In [20]:
import numpy as np
# Getting needed databases --*------*---------*------------*----------*---------------------

#List of required tables for the task
req = ["tracks", "albums"]
# Get table function
dbDict = get_tables(req) 

# QUERY IN PANDAS --*------*---------*------------*----------*----------------------*-------
rel = dbDict["tracks"].merge(dbDict["tracks"], on= 'Milliseconds', how = 'left', indicator = True)
rel = rel[rel.TrackId_x != rel.TrackId_y].loc[:,["Name_x","Name_y","Milliseconds"]]
rel



,Name_x,Name_y,Milliseconds
6,Put The Finger On You,Corcovado (Quiet Nights Of Quiet Stars),205662
11,Evil Walks,Love Is Blindness,263497
15,Night Of The Long Knives,Caso Você Queira Saber,205688
17,Spellbound,You Are,270863
20,Dog Eat Dog,Drain You,215196
...,...,...,...
4267,Confirmed Dead,Genesis,2611986
4275,Meet Kevin Johnson,Cabin Fever,2612028
4289,Cabin Fever,Meet Kevin Johnson,2612028
4340,Solomon HWV 67: The Arrival of the Queen of Sheba,You're Crazy,197135
